# Exploring the Scrambler System in `cubing_algs`

**A comprehensive guide for developers and speedcubing hobbyists**

The `cubing_algs.scrambler` module provides intelligent scramble generation for Rubik's cubes of various sizes. Unlike simple random move generators, this system creates high-quality scrambles that avoid redundant moves and ensure proper cube randomization for both practice and competition.

## What You'll Learn

- **Scramble fundamentals** - Understanding what makes a good scramble
- **Cube size adaptation** - How scrambles scale for different cube sizes (2x2 to NxN)
- **Move set generation** - Building appropriate move sets for each cube type
- **Validation algorithms** - Preventing redundant and inefficient moves
- **Specialized scrambles** - Easy cross scrambles and targeted practice
- **Performance analysis** - Understanding scramble quality and randomization
- **Practical applications** - Integration with timers, competitions, and training

## Prerequisites

- Basic understanding of Rubik's cube notation
- Familiarity with the `Algorithm` and `Move` classes (see previous notebooks)
- Understanding of cube sizes and layered moves

Let's explore the world of intelligent scramble generation!

In [1]:
# Import the scrambler and related components
from cubing_algs.scrambler import *
from cubing_algs import Algorithm
from cubing_algs.vcube import VCube
import random
from collections import Counter

# Set a seed for reproducible examples (remove for truly random scrambles)
random.seed(42)

print("=== Scrambler System Overview ===")
print("Available functions:")
print("- scramble(cube_size, iterations, inner_layers=False, right_handed=True)")
print("- scramble_easy_cross()")
print("- build_cube_move_set(cube_size, inner_layers=False, right_handed=True)")
print("- random_moves(cube_size, move_set, iterations)")
print("- is_valid_next_move(current, previous)")

# Generate some basic scrambles
scrambles_3x3 = [scramble(3) for _ in range(3)]
print(f"\nSample 3x3 scrambles:")
for i, s in enumerate(scrambles_3x3, 1):
    print(f"{i}: {s} ({len(s)} moves)")

# Different cube sizes
scramble_2x2 = scramble(2)
scramble_4x4 = scramble(4)
scramble_5x5 = scramble(5)

print(f"\nDifferent cube sizes:")
print(f"2x2: {scramble_2x2} ({len(scramble_2x2)} moves)")
print(f"4x4: {scramble_4x4} ({len(scramble_4x4)} moves)")
print(f"5x5: {scramble_5x5} ({len(scramble_5x5)} moves)")

# Handedness optimization
print(f"\n=== Handedness Optimization ===")
print("The scrambler can optimize for different handedness preferences:")

# Compare right-handed vs left-handed scrambles
rh_scramble = scramble(4, 20, right_handed=True)
lh_scramble = scramble(4, 20, right_handed=False)

print(f"Right-handed: {rh_scramble}")
print(f"Left-handed:  {lh_scramble}")

# Show move set differences
rh_moves = build_cube_move_set(4, right_handed=True)
lh_moves = build_cube_move_set(4, right_handed=False)

print(f"\nMove set differences for 4x4 cubes:")
print(f"Right-handed moves: {len(rh_moves)} total")
print(f"Left-handed moves:  {len(lh_moves)} total")
print("(Different face exclusions optimize for fingertrick preferences)")

=== Scrambler System Overview ===
Available functions:
- scramble(cube_size, iterations, inner_layers=False, right_handed=True)
- scramble_easy_cross()
- build_cube_move_set(cube_size, inner_layers=False, right_handed=True)
- random_moves(cube_size, move_set, iterations)
- is_valid_next_move(current, previous)

Sample 3x3 scrambles:
1: F U2 F' D2 R2 B' R' U R D2 B' U' B2 U2 R F2 L' U2 F' U L' F R2 B L2 (25 moves)
2: L2 U2 R' B2 D2 F R2 D2 L U R2 U' L U' F U2 B2 L2 F2 L2 U R2 F2 D2 F2 U2 L' U' R' (29 moves)
3: L' U2 R2 U L' U B U2 F' U' B' L2 U' F' D' R2 F R2 B D' R F D2 L' F L B' R (28 moves)

Different cube sizes:
2x2: U2 F2 D' F L D' F' L2 F2 D2 R (11 moves)
4x4: L' Uw R Fw2 D2 Fw R' F' L B2 Uw R2 D Rw' B Uw Rw2 F2 Uw' L' U' F Uw2 B2 U B L2 Fw2 U2 Rw F' R2 Fw' R Uw2 F' L F' R B' L2 F' R2 Fw' R2 Uw' F' (47 moves)
5x5: Uw2 F2 D B2 U F Lw' Bw Rw F Rw B' Lw F Uw Bw' Uw2 Bw2 Uw Rw' Bw' Dw2 F Rw Dw' R Uw Fw' D' B' Rw Fw' R B Uw' Bw2 L Bw Dw2 Fw U L U' Rw Dw' Rw D Fw' Rw D2 Rw2 Fw2 Rw' Uw B

## 1. Understanding Scramble Quality

Not all random move sequences make good scrambles. The `cubing_algs` scrambler implements several quality controls to ensure effective cube randomization.

### What Makes a Good Scramble?

1. **No consecutive same-face moves** - Avoids `R R'` or `R R R'` redundancies
2. **No consecutive opposite-face moves** - Prevents `R L R L` patterns that can be optimized away
3. **Appropriate length** - Enough moves to ensure full randomization
4. **Diverse move distribution** - Uses all available faces effectively

In [2]:
# Demonstrate move validation system
print("=== Move Validation System ===")

# Test the validation function
test_cases = [
    ("R", "U", True),   # Different faces - OK
    ("R", "R'", False), # Same face - NOT OK
    ("R", "L", False),  # Opposite faces - NOT OK
    ("F", "B", False),  # Opposite faces - NOT OK
    ("U", "D", False),  # Opposite faces - NOT OK
    ("F", "R", True),   # Adjacent faces - OK
    ("Rw", "R", False), # Same base face - NOT OK
    ("2R", "R", False), # Same base face - NOT OK
]

print("Move validation examples:")
for current, previous, expected in test_cases:
    result = is_valid_next_move(current, previous)
    status = "✓" if result == expected else "✗"
    print(f"{status} {previous} → {current}: {result}")

# Show what happens without validation (bad scramble)
def bad_scramble(length=20):
    """Generate a scramble without validation (for comparison)."""
    faces = ['R', 'U', 'F', 'L', 'B', 'D']
    modifiers = ['', "'", '2']
    moves = []
    for _ in range(length):
        face = random.choice(faces)
        modifier = random.choice(modifiers)
        moves.append(f"{face}{modifier}")
    return Algorithm.parse_moves(moves)

good_scramble = scramble(3, 25)
bad_scramble_example = bad_scramble(25)

print(f"\nBad scramble (no validation): {bad_scramble_example}")
print(f"Good scramble (with validation): {good_scramble}")

# Analyze scramble quality
def analyze_scramble_quality(scramble_alg):
    """Analyze the quality of a scramble."""
    moves = [str(m) for m in scramble_alg]
    faces = [m.base_move for m in scramble_alg]
    
    # Check for consecutive same faces
    same_face_violations = 0
    opposite_face_violations = 0
    
    for i in range(len(faces) - 1):
        current = faces[i]
        next_face = faces[i + 1]
        
        if current == next_face:
            same_face_violations += 1
        
        # Check opposite faces using constants
        from cubing_algs.constants import OPPOSITE_FACES
        if OPPOSITE_FACES.get(current) == next_face:
            opposite_face_violations += 1
    
    face_distribution = Counter(faces)
    
    return {
        'length': len(moves),
        'same_face_violations': same_face_violations,
        'opposite_face_violations': opposite_face_violations,
        'face_distribution': dict(face_distribution),
        'unique_faces': len(face_distribution)
    }

print(f"\nQuality Analysis:")
print(f"Good scramble: {analyze_scramble_quality(good_scramble)}")
print(f"Bad scramble: {analyze_scramble_quality(bad_scramble_example)}")

=== Move Validation System ===
Move validation examples:
✓ U → R: True
✓ R' → R: False
✓ L → R: False
✓ B → F: False
✓ D → U: False
✓ R → F: True
✓ R → Rw: False
✓ R → 2R: False

Bad scramble (no validation): B2 U U L U2 F' D2 U' U2 R' R' U L' F U D L' F F' D2 L2 B' R F B'
Good scramble (with validation): U R' B' U2 R' F' U2 F2 D2 B' D2 R F R2 F' D2 R' D2 F' R' U F L2 D2 B'

Quality Analysis:
Good scramble: {'length': 25, 'same_face_violations': 0, 'opposite_face_violations': 0, 'face_distribution': {'U': 4, 'R': 6, 'B': 3, 'F': 6, 'D': 5, 'L': 1}, 'unique_faces': 6}
Bad scramble: {'length': 25, 'same_face_violations': 4, 'opposite_face_violations': 3, 'face_distribution': {'B': 3, 'U': 7, 'L': 4, 'F': 5, 'D': 3, 'R': 3}, 'unique_faces': 6}


## 2. Cube Size Adaptation and Move Sets

The scrambler automatically adapts to different cube sizes, generating appropriate move sets and scramble lengths for each puzzle type.

### Move Set Generation by Cube Size

- **2x2**: Basic face moves only (R, U, F, etc.)
- **3x3**: Basic face moves (standard WCA scrambles)
- **4x4+**: Includes wide moves (Rw, Uw, etc.) and layered moves
- **Big cubes**: Additional inner layer moves for complete randomization

In [3]:
# Explore move set generation for different cube sizes
print("=== Cube Size Adaptation ===")

# Build move sets for different cube sizes
cube_sizes = [2, 3, 4, 5, 6, 7]

for size in cube_sizes:
    basic_moves = build_cube_move_set(size, inner_layers=False)
    inner_moves = build_cube_move_set(size, inner_layers=True) if size > 3 else basic_moves
    
    print(f"\n{size}x{size} Cube:")
    print(f"  Basic moves: {len(basic_moves)} moves")
    print(f"  With inner layers: {len(inner_moves)} moves")
    print(f"  Sample moves: {basic_moves[:12]}...")
    if size > 3:
        print(f"  Advanced moves: {[m for m in inner_moves if m not in basic_moves][:8]}...")

# Show scramble length adaptation
from cubing_algs.constants import ITERATIONS_BY_CUBE_SIZE

print(f"\n=== Scramble Length by Cube Size ===")
for size, (min_iter, max_iter) in ITERATIONS_BY_CUBE_SIZE.items():
    if size <= 6:
        sample_scramble = scramble(size)
        print(f"{size}x{size}: {min_iter}-{max_iter} moves (generated {len(sample_scramble)} moves)")

# Demonstrate the difference between basic and inner layer scrambles
print(f"\n=== Basic vs Inner Layer Scrambles (4x4) ===")
basic_4x4 = scramble(4, inner_layers=False)
advanced_4x4 = scramble(4, inner_layers=True)

print(f"Basic 4x4: {basic_4x4}")
print(f"Advanced 4x4: {advanced_4x4}")

# Analyze move type distribution
def analyze_move_types(scramble_alg):
    """Analyze the types of moves in a scramble."""
    basic_faces = set(['R', 'U', 'F', 'L', 'B', 'D'])
    wide_moves = 0
    basic_moves = 0
    inner_moves = 0
    
    for move in scramble_alg:
        if move.is_wide_move:
            wide_moves += 1
        elif move.is_inner_move:
            inner_moves += 1
        elif move.base_move in basic_faces:
            basic_moves += 1
    
    return {
        'basic': basic_moves,
        'wide': wide_moves, 
        'inner': inner_moves,
        'total': len(scramble_alg)
    }

print(f"\nMove type analysis:")
print(f"Basic 4x4: {analyze_move_types(basic_4x4)}")
print(f"Advanced 4x4: {analyze_move_types(advanced_4x4)}")

# Demonstrate handedness optimization effects
print(f"\n=== Handedness Optimization Analysis ===")

# Analyze the excluded faces for each handedness
from cubing_algs.scrambler import EXCLUDE_ODD_FACES_RH, EXCLUDE_ODD_FACES_LH

print("Face exclusions for fingertrick optimization:")
print(f"Right-handed excludes: {sorted(EXCLUDE_ODD_FACES_RH)}")
print(f"Left-handed excludes:  {sorted(EXCLUDE_ODD_FACES_LH)}")
print("Note: Both exclude D and B, but differ on L vs R")

# Generate move sets for different handedness
cube_sizes = [3, 4, 5]
print(f"\nMove set comparison by handedness:")

for size in cube_sizes:
    rh_basic = build_cube_move_set(size, inner_layers=False, right_handed=True)
    lh_basic = build_cube_move_set(size, inner_layers=False, right_handed=False)
    
    print(f"\n{size}x{size} Cube:")
    print(f"  Right-handed: {len(rh_basic)} moves")
    print(f"  Left-handed:  {len(lh_basic)} moves")
    
    if size > 3:
        # Show specific differences in move sets
        rh_set = set(rh_basic)
        lh_set = set(lh_basic)
        
        rh_only = rh_set - lh_set
        lh_only = lh_set - rh_set
        
        if rh_only:
            print(f"  Right-handed only: {sorted(list(rh_only))[:5]}...")
        if lh_only:
            print(f"  Left-handed only:  {sorted(list(lh_only))[:5]}...")

# Show specific move exclusions for odd/even cubes
print(f"\n=== Cube-specific optimizations ===")
print("The scrambler excludes certain moves on specific cube sizes:")
print("- Odd cubes: Some inner layer moves are excluded to prevent redundancy")
print("- Even cubes: Certain wide moves are excluded for efficiency")
print("- Handedness: Face exclusions vary (D,L,B for right vs D,R,B for left)")

# Practical applications
print(f"\n=== Practical Applications ===")
print("Handedness optimization is useful for:")
print("- Left-handed cubers who find R moves uncomfortable")
print("- Timer applications with user preference settings")
print("- Competition practice matching solver's handedness")
print("- Training tools targeting specific fingertrick comfort")

=== Cube Size Adaptation ===

2x2 Cube:
  Basic moves: 18 moves
  With inner layers: 18 moves
  Sample moves: ['R', "R'", 'R2', 'F', "F'", 'F2', 'U', "U'", 'U2', 'L', "L'", 'L2']...

3x3 Cube:
  Basic moves: 18 moves
  With inner layers: 18 moves
  Sample moves: ['R', "R'", 'R2', 'F', "F'", 'F2', 'U', "U'", 'U2', 'L', "L'", 'L2']...

4x4 Cube:
  Basic moves: 27 moves
  With inner layers: 45 moves
  Sample moves: ['R', "R'", 'R2', 'Rw', "Rw'", 'Rw2', 'F', "F'", 'F2', 'Fw', "Fw'", 'Fw2']...
  Advanced moves: ['2R', "2R'", '2R2', '2F', "2F'", '2F2', '2U', "2U'"]...

5x5 Cube:
  Basic moves: 36 moves
  With inner layers: 63 moves
  Sample moves: ['R', "R'", 'R2', 'Rw', "Rw'", 'Rw2', 'F', "F'", 'F2', 'Fw', "Fw'", 'Fw2']...
  Advanced moves: ['2R', "2R'", '2R2', '3R', "3R'", '3R2', '2F', "2F'"]...

6x6 Cube:
  Basic moves: 45 moves
  With inner layers: 81 moves
  Sample moves: ['R', "R'", 'R2', 'Rw', "Rw'", 'Rw2', '3Rw', "3Rw'", '3Rw2', 'F', "F'", 'F2']...
  Advanced moves: ['2R', "2R'", '2R

## 3. Specialized Scrambles and Training Tools

Beyond standard scrambles, the system provides specialized scrambling functions for targeted practice and training scenarios.

In [4]:
# Specialized scrambles for training
print("=== Specialized Scrambles ===")

# Easy cross scrambles for CFOP practice
print("Easy cross scrambles (using only F, R, B, L):")
for i in range(5):
    easy_cross = scramble_easy_cross()
    print(f"{i+1}: {easy_cross}")

# Custom training scrambles
def custom_training_scramble(move_set, length):
    """Create a custom scramble with specific moves."""
    return random_moves(3, move_set, length)

# Right-hand only scrambles (R, U moves for one-handed practice)
rh_moves = ['R', "R'", 'R2', 'U', "U'", 'U2']
rh_scrambles = [custom_training_scramble(rh_moves, 15) for _ in range(3)]
print(f"\nRight-hand only scrambles:")
for i, scramble_alg in enumerate(rh_scrambles, 1):
    print(f"{i}: {scramble_alg}")

# Trigger-heavy scrambles (for finger trick practice)
trigger_moves = ['R', "R'", 'U', "U'", 'F', "F'"]
trigger_scrambles = [custom_training_scramble(trigger_moves, 20) for _ in range(3)]
print(f"\nTrigger-heavy scrambles:")
for i, scramble_alg in enumerate(trigger_scrambles, 1):
    print(f"{i}: {scramble_alg}")

# OLL skip scrambles (avoiding certain move combinations)
def create_restricted_scramble(avoid_faces, length=20):
    """Create a scramble avoiding certain faces."""
    all_faces = ['R', 'U', 'F', 'L', 'B', 'D']
    allowed_faces = [f for f in all_faces if f not in avoid_faces]
    moves = []
    for face in allowed_faces:
        moves.extend([face, f"{face}'", f"{face}2"])
    return custom_training_scramble(moves, length)

# Avoid D moves for easier cross practice
no_d_scramble = create_restricted_scramble(['D'], 25)
print(f"\nNo D-moves scramble: {no_d_scramble}")

# Competition simulation (generate a set of scrambles)
print(f"\n=== Competition Scramble Set ===")
comp_scrambles = [scramble(3) for _ in range(5)]
for i, comp_scramble in enumerate(comp_scrambles, 1):
    print(f"Solve {i}: {comp_scramble}")
    
# Calculate average scramble statistics
total_moves = sum(len(s) for s in comp_scrambles)
avg_length = total_moves / len(comp_scrambles)
print(f"Average length: {avg_length:.1f} moves")

=== Specialized Scrambles ===
Easy cross scrambles (using only F, R, B, L):
1: R F R B R F R B L F
2: L B L B L F L B R F
3: B R B R F L B L B R
4: B L B L F L F L F L
5: L F R B L F R B R B

Right-hand only scrambles:
1: U R U2 R' U2 R' U' R U' R U R U2 R' U
2: U2 R2 U' R2 U R' U R' U R' U' R' U R2 U'
3: R2 U2 R2 U' R' U R2 U' R2 U2 R' U' R' U R

Trigger-heavy scrambles:
1: U F' U' F' U' F' R' U' R F U R U' R F U' R F' R' U'
2: F' R' U' F U' F' R U F' U' F' U' F R F R F' U R' F'
3: R U' R F' R U' R' F' U R U R U R' F U' F R' F U'

No D-moves scramble: L U F' L U F' L' U' F' U' L' B2 U' L' F2 R2 F' U2 L U' L' F L2 U' F'

=== Competition Scramble Set ===
Solve 1: L B D F U' B L2 B U2 R D L U' L2 U' F' R' B2 R' F' R2 B' U F' D2 L2
Solve 2: D' F2 U2 L F R2 F' U' B D2 L2 B R U2 F L2 U2 B L2 F U' R D2 L' U' R2 B2 L B'
Solve 3: F R' D F U' F' L2 D2 B' D B' U2 R' U B2 U' L2 F L2 D2 L2 B U2 F R2 U
Solve 4: R U' F' U R2 D2 L' F' R U2 F' D2 F2 R F' R U' L' F2 U2 R' F' D' F R2
Solve 5: D L2 D' F 

## 4. Scramble Visualization and Cube State Analysis

Understanding how scrambles affect the cube state is crucial for both learning and analysis. Let's explore scramble visualization and state analysis.

In [5]:
# Scramble visualization and analysis
print("=== Scramble Visualization and Analysis ===")

# Apply scrambles to cube and analyze the result
def analyze_scrambled_cube(scramble_alg):
    """Analyze the state after applying a scramble."""
    cube = VCube()
    cube.rotate(scramble_alg)
    
    # Check how many pieces are in their correct positions
    solved_cube = VCube()
    
    # Compare states
    scrambled_state = cube.state
    solved_state = solved_cube.state
    
    matching_facelets = sum(1 for s, r in zip(scrambled_state, solved_state) if s == r)
    total_facelets = len(solved_state)
    
    return {
        'scramble': str(scramble_alg),
        'length': len(scramble_alg),
        'matching_facelets': matching_facelets,
        'total_facelets': total_facelets,
        'scramble_percentage': (total_facelets - matching_facelets) / total_facelets * 100,
        'is_solved': cube.is_solved
    }

# Test different scramble lengths to see randomization effectiveness
print("Scramble effectiveness by length:")
test_lengths = [5, 10, 15, 20, 25, 30]

for length in test_lengths:
    test_scramble = scramble(3, length)
    analysis = analyze_scrambled_cube(test_scramble)
    print(f"{length:2d} moves: {analysis['scramble_percentage']:.1f}% scrambled "
          f"({analysis['matching_facelets']}/{analysis['total_facelets']} pieces in place)")

# Compare different cube sizes
print(f"\nScramble effectiveness by cube size:")
for size in [2, 3, 4, 5]:
    test_scramble = scramble(size)
    # For simplicity, only analyze 3x3 cubes (VCube is 3x3 only)
    if size == 3:
        analysis = analyze_scrambled_cube(test_scramble)
        print(f"{size}x{size}: {len(test_scramble)} moves, {analysis['scramble_percentage']:.1f}% scrambled")
    else:
        print(f"{size}x{size}: {len(test_scramble)} moves (visualization not available)")

# Analyze cross state after easy cross scrambles
print(f"\n=== Cross Analysis for Easy Scrambles ===")
def analyze_cross_state(scramble_alg):
    """Analyze how the cross is affected by a scramble."""
    cube = VCube()
    cube.rotate(scramble_alg)
    
    # Check D-face cross edges (positions 37, 39, 41, 43 in the state string)
    # This is a simplified cross check
    d_cross_positions = [37, 39, 41, 43]  # D-face edge positions
    d_face_color = cube.state[40]  # D-face center
    
    cross_edges_correct = 0
    for pos in d_cross_positions:
        if cube.state[pos] == d_face_color:
            cross_edges_correct += 1
    
    return cross_edges_correct

easy_scrambles = [scramble_easy_cross() for _ in range(5)]
regular_scrambles = [scramble(3, 25) for _ in range(5)]

print("Cross pieces in place after scrambles:")
print("Easy cross scrambles:")
for i, easy_scr in enumerate(easy_scrambles, 1):
    cross_pieces = analyze_cross_state(easy_scr)
    print(f"  {i}: {cross_pieces}/4 cross edges correct")

print("Regular scrambles:")
for i, reg_scr in enumerate(regular_scrambles, 1):
    cross_pieces = analyze_cross_state(reg_scr)
    print(f"  {i}: {cross_pieces}/4 cross edges correct")

# Show visual cube state (if available)
sample_scramble = scramble(3, 20)
print(f"\nSample scramble: {sample_scramble}")
cube = VCube()
cube.rotate(sample_scramble)

# Display a simple representation of the cube state
print("Cube state after scramble:")
print(f"Solved: {cube.is_solved}")
print(f"State string (first 20 chars): {cube.state[:20]}...")

# Show the scramble effect visually by comparing before/after
print(f"\nCube transformation:")
solved_cube = VCube()
print(f"Before: {solved_cube.state[:20]}... (solved)")
print(f"After:  {cube.state[:20]}... (scrambled)")

=== Scramble Visualization and Analysis ===
Scramble effectiveness by length:
 5 moves: 57.4% scrambled (23/54 pieces in place)
10 moves: 66.7% scrambled (18/54 pieces in place)
15 moves: 74.1% scrambled (14/54 pieces in place)
20 moves: 72.2% scrambled (15/54 pieces in place)
25 moves: 77.8% scrambled (12/54 pieces in place)
30 moves: 75.9% scrambled (13/54 pieces in place)

Scramble effectiveness by cube size:
2x2: 11 moves (visualization not available)
3x3: 28 moves, 72.2% scrambled
4x4: 46 moves (visualization not available)
5x5: 60 moves (visualization not available)

=== Cross Analysis for Easy Scrambles ===
Cross pieces in place after scrambles:
Easy cross scrambles:
  1: 1/4 cross edges correct
  2: 0/4 cross edges correct
  3: 1/4 cross edges correct
  4: 2/4 cross edges correct
  5: 1/4 cross edges correct
Regular scrambles:
  1: 0/4 cross edges correct
  2: 0/4 cross edges correct
  3: 0/4 cross edges correct
  4: 0/4 cross edges correct
  5: 0/4 cross edges correct

Sample 

## 5. Performance Analysis and Optimization

Understanding scramble generation performance is important for applications that need to generate many scrambles quickly, such as competition software or training apps.

In [6]:
# Performance analysis of scramble generation
import time

print("=== Scramble Generation Performance ===")

def benchmark_scramble_generation(cube_size, count=100, inner_layers=False):
    """Benchmark scramble generation performance."""
    start_time = time.perf_counter()
    
    scrambles = []
    for _ in range(count):
        scr = scramble(cube_size, inner_layers=inner_layers)
        scrambles.append(scr)
    
    end_time = time.perf_counter()
    total_time = (end_time - start_time) * 1000  # Convert to milliseconds
    
    # Calculate statistics
    total_moves = sum(len(s) for s in scrambles)
    avg_length = total_moves / count
    
    return {
        'cube_size': cube_size,
        'count': count,
        'total_time_ms': total_time,
        'avg_time_per_scramble_ms': total_time / count,
        'scrambles_per_second': count / (total_time / 1000),
        'average_length': avg_length,
        'inner_layers': inner_layers
    }

# Benchmark different cube sizes
cube_sizes = [2, 3, 4, 5, 6]
print("Performance by cube size (100 scrambles each):")
print("Size | Time/scramble | Scrambles/sec | Avg length")
print("-" * 50)

for size in cube_sizes:
    results = benchmark_scramble_generation(size, 100)
    print(f"{size:2d}x{size:<2d} | {results['avg_time_per_scramble_ms']:8.2f} ms | "
          f"{results['scrambles_per_second']:8.1f}     | {results['average_length']:6.1f}")

# Compare basic vs inner layer performance for 4x4
print(f"\n4x4 Performance comparison:")
basic_4x4 = benchmark_scramble_generation(4, 100, inner_layers=False)
advanced_4x4 = benchmark_scramble_generation(4, 100, inner_layers=True)

print(f"Basic 4x4:    {basic_4x4['avg_time_per_scramble_ms']:.2f} ms/scramble")
print(f"Advanced 4x4: {advanced_4x4['avg_time_per_scramble_ms']:.2f} ms/scramble")
print(f"Overhead:     {advanced_4x4['avg_time_per_scramble_ms'] - basic_4x4['avg_time_per_scramble_ms']:.2f} ms")

# Test bulk generation for competitions
print(f"\n=== Bulk Generation for Competitions ===")
competition_sizes = [100, 500, 1000]

for size in competition_sizes:
    start = time.perf_counter()
    comp_scrambles = [scramble(3) for _ in range(size)]
    end = time.perf_counter()
    
    total_time = (end - start) * 1000
    print(f"{size:4d} scrambles: {total_time:7.1f} ms ({total_time/size:.2f} ms each)")

# Memory usage analysis (approximate)
print(f"\n=== Memory Usage Analysis ===")
def estimate_scramble_memory(scramble_alg):
    """Estimate memory usage of a scramble."""
    # Rough estimation based on string representation
    str_size = len(str(scramble_alg)) * 1  # 1 byte per character (approximate)
    move_count = len(scramble_alg)
    # Each Move object has overhead
    move_overhead = move_count * 100  # Estimated bytes per move object
    return str_size + move_overhead

sample_scrambles = [scramble(3) for _ in range(10)]
memory_estimates = [estimate_scramble_memory(s) for s in sample_scrambles]
avg_memory = sum(memory_estimates) / len(memory_estimates)

print(f"Average scramble memory: ~{avg_memory:.0f} bytes")
print(f"1000 scrambles: ~{avg_memory * 1000 / 1024:.0f} KB")

# Quality vs performance trade-offs
print(f"\n=== Quality vs Performance Trade-offs ===")

def quick_scramble(cube_size, length):
    """Generate a scramble with minimal validation (faster but lower quality)."""
    from cubing_algs.constants import OUTER_BASIC_MOVES
    moves = []
    modifiers = ['', "'", '2']
    
    for _ in range(length):
        face = random.choice(OUTER_BASIC_MOVES)
        modifier = random.choice(modifiers)
        moves.append(f"{face}{modifier}")
    
    return Algorithm.parse_moves(moves)

# Compare quality vs performance
print("Quality vs Performance comparison (100 scrambles):")

# High quality (with validation)
start = time.perf_counter()
quality_scrambles = [scramble(3, 25) for _ in range(100)]
quality_time = (time.perf_counter() - start) * 1000

# Quick and dirty (no validation)
start = time.perf_counter()
quick_scrambles = [quick_scramble(3, 25) for _ in range(100)]
quick_time = (time.perf_counter() - start) * 1000

print(f"High quality: {quality_time:.1f} ms total ({quality_time/100:.2f} ms each)")
print(f"Quick/dirty:  {quick_time:.1f} ms total ({quick_time/100:.2f} ms each)")
print(f"Speed gain:   {quality_time/quick_time:.1f}x faster (but lower quality)")

# Analyze quality difference
quality_analysis = analyze_scramble_quality(quality_scrambles[0])
quick_analysis = analyze_scramble_quality(quick_scrambles[0])

print(f"\nQuality comparison:")
print(f"High quality violations: {quality_analysis['same_face_violations'] + quality_analysis['opposite_face_violations']}")
print(f"Quick scramble violations: {quick_analysis['same_face_violations'] + quick_analysis['opposite_face_violations']}")

=== Scramble Generation Performance ===
Performance by cube size (100 scrambles each):
Size | Time/scramble | Scrambles/sec | Avg length
--------------------------------------------------
 2x2  |     0.03 ms |  33511.5     |   10.1
 3x3  |     0.08 ms |  12956.0     |   27.6
 4x4  |     0.13 ms |   7756.4     |   47.5
 5x5  |     0.18 ms |   5690.2     |   60.0
 6x6  |     0.22 ms |   4596.9     |   80.0

4x4 Performance comparison:
Basic 4x4:    0.12 ms/scramble
Advanced 4x4: 0.12 ms/scramble
Overhead:     -0.01 ms

=== Bulk Generation for Competitions ===
 100 scrambles:     6.7 ms (0.07 ms each)
 500 scrambles:    59.1 ms (0.12 ms each)
1000 scrambles:    72.3 ms (0.07 ms each)

=== Memory Usage Analysis ===
Average scramble memory: ~2760 bytes
1000 scrambles: ~2695 KB

=== Quality vs Performance Trade-offs ===
Quality vs Performance comparison (100 scrambles):
High quality: 6.6 ms total (0.07 ms each)
Quick/dirty:  17.7 ms total (0.18 ms each)
Speed gain:   0.4x faster (but lower q

## 6. Integration with Timer Applications and Competitions

The scrambler system is designed for integration with timer applications, competition software, and training tools. Let's explore practical integration patterns.

In [7]:
# Integration examples for timer applications
print("=== Timer Application Integration ===")

class SimpleTimer:
    """Example timer application using the scrambler system."""
    
    def __init__(self, right_handed=True):
        self.right_handed = right_handed
        self.current_scramble = None
        self.scramble_history = []
        self.solve_times = []
    
    def new_scramble(self, cube_size=3):
        """Generate a new scramble for timing."""
        self.current_scramble = scramble(cube_size, right_handed=self.right_handed)
        self.scramble_history.append(self.current_scramble)
        return self.current_scramble
    
    def get_scramble_text(self):
        """Get formatted scramble text for display."""
        if self.current_scramble:
            return str(self.current_scramble)
        return "No scramble generated"
    
    def start_solve(self):
        """Start timing a solve."""
        handedness = "right-handed" if self.right_handed else "left-handed"
        print(f"Scramble ({handedness}): {self.get_scramble_text()}")
        print("Starting solve... (timer would start here)")
    
    def generate_session_scrambles(self, count=5, cube_size=3):
        """Generate a session of scrambles for practice."""
        session_scrambles = []
        for i in range(count):
            scr = scramble(cube_size, right_handed=self.right_handed)
            session_scrambles.append(scr)
        return session_scrambles

# Demo timer usage with handedness options
print("Timer Application Demo:")

# Generate scrambles for both handedness preferences
rh_timer = SimpleTimer(right_handed=True)
lh_timer = SimpleTimer(right_handed=False)

print("Right-handed practice session:")
rh_session = rh_timer.generate_session_scrambles(3)
for i, scr in enumerate(rh_session, 1):
    print(f"{i}: {scr}")

print("\nLeft-handed practice session:")
lh_session = lh_timer.generate_session_scrambles(3)
for i, scr in enumerate(lh_session, 1):
    print(f"{i}: {scr}")

# Competition scramble generation
print(f"\n=== Competition Integration ===")

class CompetitionScrambler:
    """Example competition scramble generator."""
    
    def __init__(self):
        self.event_configs = {
            '2x2': {'cube_size': 2},
            '3x3': {'cube_size': 3},
            '4x4': {'cube_size': 4, 'inner_layers': False},
            '5x5': {'cube_size': 5, 'inner_layers': False},
            '6x6': {'cube_size': 6, 'inner_layers': False},
            '7x7': {'cube_size': 7, 'inner_layers': False},
        }
    
    def generate_round_scrambles(self, event, round_name, count=5, right_handed=True):
        """Generate scrambles for a competition round."""
        if event not in self.event_configs:
            raise ValueError(f"Unknown event: {event}")
        
        config = self.event_configs[event]
        scrambles = []
        
        for i in range(count):
            scr = scramble(config['cube_size'], 
                          inner_layers=config.get('inner_layers', False),
                          right_handed=right_handed)
            scrambles.append({
                'number': i + 1,
                'scramble': str(scr),
                'algorithm': scr,
                'event': event,
                'round': round_name,
                'handedness': 'right-handed' if right_handed else 'left-handed'
            })
        
        return scrambles
    
    def export_scrambles(self, event, round_name, count=5, right_handed=True):
        """Export scrambles in competition format."""
        scrambles = self.generate_round_scrambles(event, round_name, count, right_handed)
        
        handedness = "right-handed" if right_handed else "left-handed"
        output = []
        output.append(f"Event: {event}")
        output.append(f"Round: {round_name}")
        output.append(f"Optimization: {handedness}")
        output.append(f"Generated: {count} scrambles")
        output.append("-" * 50)
        
        for scr_data in scrambles:
            output.append(f"{scr_data['number']:2d}. {scr_data['scramble']}")
        
        return "\n".join(output)

# Demo competition usage with handedness
comp_scrambler = CompetitionScrambler()

# Generate scrambles for different events
events = ['3x3', '4x4']
for event in events:
    print(f"\n{event} Competition Scrambles:")
    rh_round_scrambles = comp_scrambler.generate_round_scrambles(event, "Final", 2, right_handed=True)
    lh_round_scrambles = comp_scrambler.generate_round_scrambles(event, "Final", 2, right_handed=False)
    
    print("Right-handed optimized:")
    for scr_data in rh_round_scrambles:
        print(f"  {scr_data['number']}: {scr_data['scramble']}")
    
    print("Left-handed optimized:")
    for scr_data in lh_round_scrambles:
        print(f"  {scr_data['number']}: {scr_data['scramble']}")

# Training application integration
print(f"\n=== Training Application Integration ===")

class TrainingApp:
    """Example training application with specialized scrambles."""
    
    def __init__(self, right_handed=True):
        self.right_handed = right_handed
        self.training_modes = {
            'easy_cross': self._easy_cross_session,
            'one_handed': self._one_handed_session,
            'trigger_practice': self._trigger_session,
            'no_rotation': self._no_rotation_session,
            'handedness_optimized': self._handedness_optimized_session
        }
    
    def _easy_cross_session(self, count=10):
        """Generate easy cross scrambles for CFOP practice."""
        return [scramble_easy_cross() for _ in range(count)]
    
    def _one_handed_session(self, count=10):
        """Generate R,U only scrambles for OH practice."""
        oh_moves = ['R', "R'", 'R2', 'U', "U'", 'U2']
        return [random_moves(3, oh_moves, 15) for _ in range(count)]
    
    def _trigger_session(self, count=10):
        """Generate trigger-heavy scrambles."""
        trigger_moves = ['R', "R'", 'U', "U'", 'F', "F'"]
        return [random_moves(3, trigger_moves, 20) for _ in range(count)]
    
    def _no_rotation_session(self, count=10):
        """Generate scrambles without rotations for practice."""
        return [scramble(3, 25, right_handed=self.right_handed) for _ in range(count)]
    
    def _handedness_optimized_session(self, count=10):
        """Generate scrambles optimized for user's handedness."""
        return [scramble(3, 25, right_handed=self.right_handed) for _ in range(count)]
    
    def start_training(self, mode, duration_minutes=10):
        """Start a training session."""
        if mode not in self.training_modes:
            available = ', '.join(self.training_modes.keys())
            raise ValueError(f"Unknown mode: {mode}. Available: {available}")
        
        # Estimate scrambles needed (assuming 30 seconds per solve average)
        estimated_solves = duration_minutes * 2
        training_scrambles = self.training_modes[mode](estimated_solves)
        
        handedness = "right-handed" if self.right_handed else "left-handed"
        return {
            'mode': mode,
            'duration': duration_minutes,
            'scrambles': training_scrambles,
            'count': len(training_scrambles),
            'handedness': handedness
        }

# Demo training app with handedness
print("Training modes with handedness optimization:")
for handedness, right_handed in [("right-handed", True), ("left-handed", False)]:
    training_app = TrainingApp(right_handed=right_handed)
    session = training_app.start_training('handedness_optimized', 5)  # 5-minute session
    print(f"  {handedness.title()}: {session['count']} scrambles")
    print(f"    Sample: {session['scrambles'][0]}")

# Scramble validation and export
print(f"\n=== Scramble Export and Validation ===")

def export_scrambles_to_file(scrambles, filename, event="3x3", right_handed=True):
    """Export scrambles to a file format."""
    handedness = "right-handed" if right_handed else "left-handed"
    lines = []
    lines.append(f"Event: {event}")
    lines.append(f"Optimization: {handedness}")
    lines.append(f"Total scrambles: {len(scrambles)}")
    lines.append(f"Generated on: {time.strftime('%Y-%m-%d %H:%M:%S')}")
    lines.append("")
    
    for i, scr in enumerate(scrambles, 1):
        lines.append(f"{i:3d}. {scr}")
    
    return '\n'.join(lines)

# Demo export with handedness
sample_scrambles_rh = [scramble(3, right_handed=True) for _ in range(5)]
sample_scrambles_lh = [scramble(3, right_handed=False) for _ in range(5)]

rh_exported = export_scrambles_to_file(sample_scrambles_rh, "rh_scrambles.txt", right_handed=True)
lh_exported = export_scrambles_to_file(sample_scrambles_lh, "lh_scrambles.txt", right_handed=False)

print("Right-handed export format:")
print('\n'.join(rh_exported.split('\n')[:8]))
print("\nLeft-handed export format:")
print('\n'.join(lh_exported.split('\n')[:8]))

=== Timer Application Integration ===
Timer Application Demo:
Right-handed practice session:
1: U2 L' F2 D' B L' D' F U F' U2 B L' D R2 B D R' U' F2 D R2 B2 D2 L D2
2: F2 R D' L' F U' F U' R' U B' L' B2 R' B R U' B' L B R' D2 B2 D B2 U' L2 B'
3: U2 B2 D2 L2 D' B' R' U' B2 R' B' U2 L2 U2 B R B' U2 L' F L2 B' R' D L' U B2

Left-handed practice session:
1: U' L2 B2 D2 B U2 F R D B' U' B R2 B2 L D2 L F U2 B' R2 B' U2 B' U' R2 F L' F'
2: U' R2 D' L B L2 U' R' U2 R' F D2 L' D B R2 U2 B2 R F2 D2 L B R2 D R F U' R'
3: F U L2 U2 L F' L' D2 B L F' U2 F' U B U2 B2 L' U F2 L2 F U2 R' D L2 B

=== Competition Integration ===

3x3 Competition Scrambles:
Right-handed optimized:
  1: L2 U F D' R' F L2 F L' U L2 U B R' U L B' D L2 B2 U2 F U2 B L2
  2: U2 L' D' R2 F' R' D L2 F2 R2 D L' B' R U B2 U' F2 D' B' L B2 D' B R2
Left-handed optimized:
  1: F2 U2 R U2 F' U' F' D F2 D R F2 D B U2 F U' F R2 U B2 U2 F L' D' F2 L2 U2 R'
  2: F2 L F' R2 D' B' U2 F L U' B' D' B R' F2 R' U2 R F' L2 U' L2 B2 U2 B2 U2 F R


## 7. Advanced Scrambling Techniques and Customization

For advanced users and specialized applications, the scrambler system can be customized and extended for specific needs.

In [8]:
# Advanced scrambling techniques
print("=== Advanced Scrambling Techniques ===")

# Custom move set creation
def create_custom_move_set(faces, include_primes=True, include_doubles=True, 
                          wide_moves=False, layered_moves=None):
    """Create a custom move set with specific constraints."""
    moves = []
    
    for face in faces:
        moves.append(face)
        if include_primes:
            moves.append(f"{face}'")
        if include_doubles:
            moves.append(f"{face}2")
        
        if wide_moves:
            moves.extend([f"{face}w", f"{face}w'", f"{face}w2"])
        
        if layered_moves:
            for layer in layered_moves:
                moves.extend([f"{layer}{face}", f"{layer}{face}'", f"{layer}{face}2"])
    
    return moves

# Create specialized move sets
print("Custom move sets:")

# Beginner-friendly (no L, B, D)
beginner_faces = ['R', 'U', 'F']
beginner_moves = create_custom_move_set(beginner_faces)
print(f"Beginner moves: {beginner_moves}")

# One-handed solving (R, U, M only)
oh_faces = ['R', 'U']
oh_moves = create_custom_move_set(oh_faces) + ['M', "M'", 'M2']
print(f"One-handed moves: {oh_moves}")

# Fewest moves challenge (more diverse)
fmc_faces = ['R', 'U', 'F', 'L', 'B', 'D']
fmc_moves = create_custom_move_set(fmc_faces, wide_moves=True, layered_moves=['2'])
print(f"FMC moves (sample): {fmc_moves[:15]}... ({len(fmc_moves)} total)")

# State-aware scrambling
print(f"\n=== State-Aware Scrambling ===")

class StateAwareScrambler:
    """Scrambler that considers current cube state."""
    
    def __init__(self):
        self.cube = VCube()
        self.move_history = []
    
    def reset_cube(self):
        """Reset to solved state."""
        self.cube = VCube()
        self.move_history = []
    
    def apply_move(self, move_str):
        """Apply a move and track history."""
        from cubing_algs import Algorithm
        move_alg = Algorithm([move_str])
        self.cube.rotate(move_alg)
        self.move_history.append(move_str)
    
    def get_legal_moves(self, move_set):
        """Get legal moves that don't undo recent moves."""
        if len(self.move_history) == 0:
            return move_set
        
        last_move = self.move_history[-1]
        legal_moves = []
        
        for move in move_set:
            if is_valid_next_move(move, last_move):
                legal_moves.append(move)
        
        return legal_moves
    
    def generate_adaptive_scramble(self, target_length=25):
        """Generate a scramble that adapts to cube state."""
        self.reset_cube()
        basic_moves = ['R', "R'", 'R2', 'U', "U'", 'U2', 
                      'F', "F'", 'F2', 'L', "L'", 'L2',
                      'B', "B'", 'B2', 'D', "D'", 'D2']
        
        scramble_moves = []
        
        for _ in range(target_length):
            legal_moves = self.get_legal_moves(basic_moves)
            if not legal_moves:
                # Fallback if no legal moves (shouldn't happen)
                legal_moves = basic_moves
            
            next_move = random.choice(legal_moves)
            self.apply_move(next_move)
            scramble_moves.append(next_move)
        
        return Algorithm.parse_moves(scramble_moves)

# Demo state-aware scrambling
state_scrambler = StateAwareScrambler()
adaptive_scramble = state_scrambler.generate_adaptive_scramble(20)
print(f"Adaptive scramble: {adaptive_scramble}")

# Compare with regular scramble
regular_scramble = scramble(3, 20)
print(f"Regular scramble:  {regular_scramble}")

# Analyze quality difference
adaptive_quality = analyze_scramble_quality(adaptive_scramble)
regular_quality = analyze_scramble_quality(regular_scramble)

print(f"Quality comparison:")
print(f"Adaptive violations: {adaptive_quality['same_face_violations'] + adaptive_quality['opposite_face_violations']}")
print(f"Regular violations:  {regular_quality['same_face_violations'] + regular_quality['opposite_face_violations']}")

# Weighted scrambling (some moves more likely)
print(f"\n=== Weighted Scrambling ===")

def weighted_scramble(move_weights, length=25):
    """Generate scramble with weighted move selection."""
    moves = list(move_weights.keys())
    weights = list(move_weights.values())
    
    scramble_moves = []
    previous_move = None
    
    for _ in range(length):
        # Choose move based on weights
        while True:
            chosen_move = random.choices(moves, weights=weights)[0]
            if previous_move is None or is_valid_next_move(chosen_move, previous_move):
                break
        
        scramble_moves.append(chosen_move)
        previous_move = chosen_move
    
    return Algorithm.parse_moves(scramble_moves)

# Create a weighted move set (favor R and U moves)
move_weights = {
    'R': 3, "R'": 3, 'R2': 2,     # High weight for R moves
    'U': 3, "U'": 3, 'U2': 2,     # High weight for U moves
    'F': 1, "F'": 1, 'F2': 1,     # Lower weight for other moves
    'L': 1, "L'": 1, 'L2': 1,
    'B': 1, "B'": 1, 'B2': 1,
    'D': 1, "D'": 1, 'D2': 1,
}

weighted_scr = weighted_scramble(move_weights, 25)
print(f"Weighted scramble (favoring R,U): {weighted_scr}")

# Analyze move distribution
def analyze_move_distribution(scramble_alg):
    """Analyze distribution of move types in scramble."""
    face_counts = Counter(move.base_move for move in scramble_alg)
    return dict(face_counts)

weighted_dist = analyze_move_distribution(weighted_scr)
regular_dist = analyze_move_distribution(regular_scramble)

print(f"Weighted distribution: {weighted_dist}")
print(f"Regular distribution:  {regular_dist}")

# Scramble validation and repair
print(f"\n=== Scramble Validation and Repair ===")

def validate_and_repair_scramble(scramble_alg):
    """Validate scramble and repair violations."""
    moves = [str(m) for m in scramble_alg]
    repaired_moves = []
    
    from cubing_algs.constants import OUTER_BASIC_MOVES
    all_moves = []
    for face in OUTER_BASIC_MOVES:
        all_moves.extend([face, f"{face}'", f"{face}2"])
    
    for i, move in enumerate(moves):
        if i == 0:
            repaired_moves.append(move)
        else:
            # Check if current move is valid after previous
            if is_valid_next_move(move, repaired_moves[-1]):
                repaired_moves.append(move)
            else:
                # Find a replacement move
                replacement_found = False
                for replacement in all_moves:
                    if is_valid_next_move(replacement, repaired_moves[-1]):
                        repaired_moves.append(replacement)
                        replacement_found = True
                        break
                
                if not replacement_found:
                    # Skip this move if no valid replacement
                    continue
    
    return Algorithm.parse_moves(repaired_moves)

# Create a bad scramble and repair it
bad_moves = ["R", "R'", "R", "L", "R", "R2"]  # Has violations
bad_scramble_alg = Algorithm.parse_moves(bad_moves)
repaired = validate_and_repair_scramble(bad_scramble_alg)

print(f"Bad scramble:     {bad_scramble_alg}")
print(f"Repaired scramble: {repaired}")
print(f"Original violations: {analyze_scramble_quality(bad_scramble_alg)['same_face_violations'] + analyze_scramble_quality(bad_scramble_alg)['opposite_face_violations']}")
print(f"Repaired violations: {analyze_scramble_quality(repaired)['same_face_violations'] + analyze_scramble_quality(repaired)['opposite_face_violations']}")

=== Advanced Scrambling Techniques ===
Custom move sets:
Beginner moves: ['R', "R'", 'R2', 'U', "U'", 'U2', 'F', "F'", 'F2']
One-handed moves: ['R', "R'", 'R2', 'U', "U'", 'U2', 'M', "M'", 'M2']
FMC moves (sample): ['R', "R'", 'R2', 'Rw', "Rw'", 'Rw2', '2R', "2R'", '2R2', 'U', "U'", 'U2', 'Uw', "Uw'", 'Uw2']... (54 total)

=== State-Aware Scrambling ===
Adaptive scramble: B R D' B R U' R U B D F2 L2 F2 R D R' D2 R' F2 D'
Regular scramble:  U F' D B R' U L D2 B2 D' R' U2 B2 U2 L2 D R2 F2 D B2
Quality comparison:
Adaptive violations: 0
Regular violations:  0

=== Weighted Scrambling ===
Weighted scramble (favoring R,U): R' D R B' R U R2 U R U' R' U R' F U' R B D2 B' U' L U' B2 L2 F'
Weighted distribution: {'R': 8, 'D': 2, 'B': 4, 'U': 7, 'F': 2, 'L': 2}
Regular distribution:  {'U': 4, 'F': 2, 'D': 5, 'B': 4, 'R': 3, 'L': 2}

=== Scramble Validation and Repair ===
Bad scramble:     R R' R L R R2
Repaired scramble: R F R F R F
Original violations: 5
Repaired violations: 0


## 8. Best Practices and Guidelines

Based on our comprehensive exploration, here are key recommendations for using the scrambler system effectively.

### When to Use Each Scramble Type

- **`scramble(3)`**: Standard 3x3 competition scrambles
- **`scramble(cube_size)`**: Appropriate scrambles for any cube size
- **`scramble_easy_cross()`**: CFOP cross practice and beginner training
- **Custom move sets**: Specialized training (one-handed, trigger practice, etc.)
- **Weighted scrambles**: Research and algorithm development

### Performance Considerations

1. **Batch generation**: Generate multiple scrambles at once for better performance
2. **Move set caching**: Reuse move sets for repeated scramble generation
3. **Quality vs speed**: Use validation for competition scrambles, skip for bulk training
4. **Memory management**: For large-scale applications, consider scramble disposal

### Integration Guidelines

```python
# Standard competition pattern
def generate_competition_round(event, count=5):
    cube_size_map = {'2x2': 2, '3x3': 3, '4x4': 4, '5x5': 5}
    size = cube_size_map[event]
    return [scramble(size) for _ in range(count)]

# Training session pattern  
def generate_training_session(mode, duration_minutes):
    moves_per_minute = 2  # Assume 30s average solve
    count = duration_minutes * moves_per_minute
    
    if mode == 'easy_cross':
        return [scramble_easy_cross() for _ in range(count)]
    elif mode == 'standard':
        return [scramble(3) for _ in range(count)]
    # Add more modes as needed
```

### Quality Assurance

- Always validate scrambles for official competitions
- Test scramble quality with the analysis functions provided
- Monitor performance metrics for large-scale generation
- Consider cube state analysis for training effectiveness

## 9. Summary and Next Steps

The `cubing_algs.scrambler` module provides a sophisticated, intelligent scramble generation system that serves both competitive and training needs in the speedcubing community.

### What We Covered

1. **Scramble Quality**: Understanding validation rules and what makes effective scrambles
2. **Cube Size Adaptation**: Automatic move set generation for 2x2 through NxN cubes
3. **Handedness Optimization**: Scramble generation optimized for right-handed or left-handed cubers
4. **Specialized Scrambles**: Easy cross training and custom move set scrambles
5. **Visualization and Analysis**: Tools for understanding scramble effectiveness
6. **Performance Optimization**: Benchmarking and optimization techniques for bulk generation
7. **Integration Patterns**: Examples for timer apps, competition software, and training tools
8. **Advanced Techniques**: Custom move sets, weighted selection, and state-aware generation
9. **Best Practices**: Guidelines for effective scrambler usage

### Key Benefits

- **Intelligent validation**: Prevents redundant moves and poor scramble quality
- **Automatic adaptation**: Scales appropriately for different cube sizes
- **Handedness support**: Optimizes move sets for both right and left-handed cubers
- **Performance optimized**: Fast generation suitable for real-time applications
- **Highly customizable**: Extensible for specialized training and research needs
- **Competition ready**: Meets standards for official competition scrambles

### Handedness Optimization Features

The scrambler system recognizes that different handedness affects fingertrick comfort and execution:

- **Right-handed optimization**: Excludes D, L, B moves from certain positions for better fingertricks
- **Left-handed optimization**: Excludes D, R, B moves instead to match left-handed preferences  
- **Configurable parameters**: Use `right_handed=False` in `scramble()` and `build_cube_move_set()`
- **Timer integration**: Perfect for personalized training applications and user preferences
- **Competition use**: Allows generating scrambles optimized for competitor handedness

### Real-World Applications

- **Timer applications**: Real-time scramble generation with handedness preferences
- **Competition software**: Bulk generation for official competitions  
- **Training tools**: Specialized scrambles for targeted skill development
- **Accessibility features**: Left-handed optimization for inclusive design
- **Research platforms**: Custom scrambles for algorithm development and analysis
- **Educational tools**: Progressive scrambles for learning and instruction

### Next Steps

- Integrate the scrambler with VCube visualization for visual scramble analysis
- Experiment with handedness optimization for different solving methods
- Build timer applications using the performance-optimized generation patterns  
- Research the relationship between handedness and solving performance
- Explore the effectiveness of left-handed optimization in real practice sessions
- Develop specialized scrambles for advanced solving methods beyond CFOP

The scrambler system forms a crucial component of the `cubing_algs` ecosystem, enabling everything from casual practice to serious competition preparation. With handedness optimization support, it now serves an even broader range of speedcubers, making the system more inclusive and accessible. Whether you're building a timer app, organizing competitions, conducting cubing research, or simply want optimized practice scrambles for your handedness preference, this system provides the foundation for high-quality scramble generation.